In [2]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00


In [3]:
file_name = 'Insert file name here'
# For Kaggle use the following line
file='/kaggle/input/20k-cleaned/'+file_name

In [4]:
from deep_translator import GoogleTranslator
import pandas as pd
from tqdm import tqdm

df = pd.read_csv(file)
df['translated_text']=''

In [5]:
df.head()

,cleaned_text,emojis,translated_text
0,یہ ٹائم کی پابندی ہٹائی جائے,😎,
1,یہ ٹائم لائن جام کیوں ہے ؟ ایسا صرف میری طرف ہ...,😩,
2,یہ ٹائیگر فورس کو کس کتے کام پہ لگا دیا ہے منقول,😆,
3,یہ ٹائیگر کا ابو ہےشیرررر اور جو مرضی کرلو باپ...,😂🥳,
4,یہ ٹٹو انشاء اللہ بہت جلد جانے والے ہیں,🍃,


In [6]:
import pandas as pd
from deep_translator import GoogleTranslator
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

# Assume df is your DataFrame and it's already loaded
source_column = 'cleaned_text'
target_column = 'translated_text'



translator = GoogleTranslator(source='auto', target='en')

def translate_texts(dataframe, source_column, target_column, num_workers=5, start_index=0):
    error_occurred = [False]

    def translate_row(index, text):
        if error_occurred[0]:  # Skip if an error occurred
            return index, None, "Skipped due to prior error"
        try:
            translated_text = translator.translate(text)
            return index, translated_text, None
        except Exception as e:
            error_occurred[0] = True
            return index, None, str(e)

    tasks_to_process = [
        (index, row[source_column]) for index, row in dataframe[dataframe.index >= start_index].iterrows()
        if pd.isna(row[target_column]) or row[target_column] == '' or row[target_column] == 'Translation Error'
    ]

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_index = {
            executor.submit(translate_row, index, text): index for index, text in tasks_to_process
        }

        # Wrap as_completed with tqdm for a progress bar
        progress = tqdm(as_completed(future_to_index), total=len(future_to_index), desc="Translating", unit="rows")

        for future in progress:
            index = future_to_index[future]
            _, translated_text, error = future.result()
            
            if error and error != "Skipped due to prior error":
                print(f"Error at index {index}: {error}")
                break  # Stop further processing if an actual error occurred
            
            if not error:  # Only update DataFrame if no error
                dataframe.at[index, target_column] = translated_text

    dataframe.to_csv(file_name, index=False)
    print("Progress saved.")

In [7]:
if target_column not in df.columns:
    df[target_column] = pd.NA
# Example usage
translate_texts(df, source_column, target_column, num_workers=20,start_index=0)

Translating:   0%|          | 0/20000 [00:00<?, ?rows/s]

Progress saved.


In [8]:
'''
# Reload the DataFrame
df = pd.read_csv('/kaggle/input/cleaned4/c4.csv',low_memory=False)

# Filter rows that need retranslation (Translation Error or empty)
rows_to_retranslate = df[df['translated_text'].isna() | (df['translated_text'] == '') | (df['translated_text'] == 'Translation Error')]

# If there are rows to retranslate, run the translation function again
if not rows_to_retranslate.empty:
    print(f"Retranslating {len(rows_to_retranslate)} rows...")
    # Here, update the translate_texts function or call it appropriately
    translate_texts(df, 'cleaned_text', 'translated_text', num_workers=20)
else:
    print("No rows require retranslation.")
'''

'\n# Reload the DataFrame\ndf = pd.read_csv(\'/kaggle/input/cleaned4/c4.csv\',low_memory=False)\n\n# Filter rows that need retranslation (Translation Error or empty)\nrows_to_retranslate = df[df[\'translated_text\'].isna() | (df[\'translated_text\'] == \'\') | (df[\'translated_text\'] == \'Translation Error\')]\n\n# If there are rows to retranslate, run the translation function again\nif not rows_to_retranslate.empty:\n    print(f"Retranslating {len(rows_to_retranslate)} rows...")\n    # Here, update the translate_texts function or call it appropriately\n    translate_texts(df, \'cleaned_text\', \'translated_text\', num_workers=20)\nelse:\n    print("No rows require retranslation.")\n'